In [ ]:
import pandas as pd
import numpy as np
import joblib
import holidays
import optuna
import plotly.graph_objects as go

# Carregar arquivos

In [ ]:
# Carrega o modelo qtimo
lgbm_otimo = joblib.load('../models/lgbm_otimizado.joblib')
# Carrega a lista de features
features_do_modelo = joblib.load('../models/lista_de_features.joblib')


# Carregar datasets

In [ ]:
# Carregandod datasets
df_train = pd.read_csv('../data/raw/train.csv') 
df_test = pd.read_csv('../data/raw/test.csv')

# Guardar os IDs do teste para o arquivo final
ids_teste = df_test['id']

# Juntar os dataframes
df_combinado = pd.concat([df_train.reset_index(), df_test], ignore_index=True)

# Engenharia de Feature

In [ ]:
# Transformações de data
df_combinado['date'] = pd.to_datetime(df_combinado['date'])
df_combinado = df_combinado.set_index('date')
df_combinado.sort_index(inplace=True)

# Features de Calendário e Feriados
df_combinado['mes'] = df_combinado.index.month
df_combinado['dia_da_semana'] = df_combinado.index.dayofweek
df_combinado['dia_do_ano'] = df_combinado.index.dayofyear
df_combinado['semana_do_ano'] = df_combinado.index.isocalendar().week.astype(int)
df_combinado['ano'] = df_combinado.index.year
feriados_br = holidays.Brazil(years=range(2013, 2019))
df_combinado['e_feriado'] = df_combinado.index.to_series().apply(lambda x: feriados_br.get(x)).notna().astype(int)

# Features de Lag e Janela Móvel
df_combinado['vendas_lag_7d'] = df_combinado.groupby(['store', 'item'])['sales'].shift(7)
df_combinado['media_vendas_7d'] = df_combinado.groupby(['store', 'item'])['sales'].transform(
    lambda x: x.shift(1).rolling(window=7, min_periods=1).mean())
df_combinado['vendas_lag_15d'] = df_combinado.groupby(['store', 'item'])['sales'].shift(15)
df_combinado['vendas_lag_30d'] = df_combinado.groupby(['store', 'item'])['sales'].shift(30)
df_combinado['media_vendas_30d'] = df_combinado.groupby(['store', 'item'])['sales'].transform(
    lambda x: x.shift(1).rolling(window=30, min_periods=1).mean()
)

In [ ]:
df_combinado

# Realizando previsão

In [ ]:
# Separar o DataFrame de Teste Final
df_teste_final = df_combinado[df_combinado['sales'].isna()]

In [ ]:
df_teste_final

# Garantir que as colunas estão na ordem correta usando a lista que salvamos
x_teste_final = df_teste_final[features_do_modelo]

nome_arquivo = 'df_ML_test.csv'
x_teste_final.to_csv(nome_arquivo)

In [ ]:
# Fazer as previsões com o modelo carregado
previsoes_finais = lgbm_otimo.predict(x_teste_final)
previsoes_finais[previsoes_finais < 0] = 0 

# Vizualizando previsão

In [ ]:
# Agregando o total de vendas reais por dia
df_train['date'] = pd.to_datetime(df_train['date'])
df_train = df_train.set_index('date')
vendas_reais_diarias = df_train['sales'].resample('D').sum()

In [ ]:
# Preparar os dados de previsão
previsoes_com_data = pd.Series(previsoes_finais, index=x_teste_final.index)
# Agregamos as previsões por dia, para ter a mesma granularidade dos dados reais
previsoes_diarias = previsoes_com_data.resample('D').sum()

In [ ]:
previsoes_com_data

In [ ]:
# 3. Criar a figura
fig = go.Figure()

# Adicionar a linha de Vendas Reais (Histórico)
fig.add_trace(go.Scatter(
    x=vendas_reais_diarias.index,
    y=vendas_reais_diarias.values,
    mode='lines',
    name='Vendas Reais (Histórico)',
    line=dict(color='blue')
))

# Adicionar a linha de Previsões
fig.add_trace(go.Scatter(
    x=previsoes_diarias.index,
    y=previsoes_diarias.values,
    mode='lines',
    name='Previsões (2018)',
    line=dict(color='red', dash='dash')
))

# 4. Customizar o layout para um visual profissional
fig.update_layout(
    title='Histórico de Vendas (2013-2017) vs. Previsões (2018)',
    yaxis_title='Total de Vendas',
    legend_title='Legenda'
)

# Exibir o gráfico interativo
fig.show()